In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from IPython.display import clear_output

!pip install underthesea

!pip install -U ipywidgets
clear_output()

In [2]:
! cat /kaggle/input/viwik18/dataset/viwik18_* > viwik18.txt

In [3]:
# Path to your ViWik dataset file
viwik_file = '/kaggle/working/viwik18.txt'

with open (viwik_file) as file: 
    corpus = file.read()

In [4]:
len(corpus)

725975404

In [42]:
sample_text = corpus[:10000000]

In [45]:
import unicodedata
import pandas as pd
import re
from underthesea import word_tokenize

stop_words = pd.read_csv('/kaggle/input/stopwords/vietnamese-stopwords-dash.txt', header=None)

def clean_text(text):
    # Convert to lowercase
    text = text.lower() 
    
    # Normalize Unicode characters
    text = unicodedata.normalize('NFKC', text)
    
    # Remove numbers
    text = re.sub(r'[^a-zA-ZÀ-ỹ ]+', '', text)
    
    # Tokenize the cleaned sentence
    tokenized_sentence = word_tokenize(text, format='text').split()  # List of words
    cleaned_sentence = ' '.join(word for word in tokenized_sentence if word not in stop_words[0].tolist())
    
    return cleaned_sentence  # Return list of tokenized sentences
#     return tokenized_text 

In [ ]:
cleaned_text = clean_text(sample_text)

In [ ]:
from collections import Counter

# Count word frequencies
word_counts = Counter(cleaned_text.split())
print(word_counts)

# Set the threshold for minimum frequency
threshold = 2

# Filter words based on frequency
filtered_words = [word for word in cleaned_text.split() if word_counts[word] > threshold]
filtered_words[:10]

In [63]:
import gensim
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

# Create a Word2Vec model
model = Word2Vec([filtered_words], vector_size=100, window=3, min_count=5, workers=4)

# Save the model
model.save("viwik_word2vec.model")

In [ ]:
# Load the model (you can do this in a separate script if needed)
loaded_model = Word2Vec.load("viwik_word2vec.model")

print("keys from KeyedVector: ", loaded_model.wv.index_to_key)

In [73]:
# Get the vector for a word
# vector = loaded_model.wv['nông_nghiệp']  # Replace 'từ' with any Vietnamese word

# Find most similar words
similar_words = loaded_model.wv.most_similar('sài_gòn')  # Replace 'từ' with any Vietnamese word

print(similar_words)

[('thành_phố', 0.9219205975532532), ('thụy_điển', 0.8914080262184143), ('kinh_tế', 0.8684271574020386), ('đông', 0.8352999091148376), ('phát_triển', 0.8321718573570251), ('chữ', 0.8308694362640381), ('bắc', 0.8299391269683838), ('hai', 0.8257130980491638), ('khu_vực', 0.8226958513259888), ('california', 0.8199417591094971)]


In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Get the embeddings and corresponding words
words = list(model.wv.key_to_index)
embeddings = model.wv[words]

# Use t-SNE to reduce dimensionality to 2D
tsne = TSNE(n_components=2, random_state=42)
embeddings_2d = tsne.fit_transform(embeddings)

# Plot
plt.figure(figsize=(20, 20))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1])

for i, word in enumerate(words):
    plt.annotate(word, (embeddings_2d[i, 0], embeddings_2d[i, 1]), fontsize=9)

plt.title("2D Visualization of Word Embeddings")
plt.show()
